rate limit 
    - If there is a live match, once every hour
    - If there is NO live match, once per day only
so i need to save reponse locally 
https://rapidapi.com/Creativesdev/api/free-api-live-football-data/playground/apiendpoint_20d92b01-b539-43d8-a8df-05c5fad5cd71

In [ ]:
#import libs 
import os 
import pandas as pd 
import json
import dotenv from load_dotenv
import requests

SyntaxError: invalid syntax (3489331078.py, line 5)

In [25]:
#load API key, host and parameters  into py file  
# TO LOAD ENVIRONMENT VARS FROM .env file INTO THE SESSION
#load_dotenv()

#api_key = os.getenv("API_KEY")
#api_key = "d3335995damsh29951d1696ba593p1c1ad1jsn963fdedea7cd"
#api_host = "api-football-v1.p.rapidapi.com"
league_id = 47
#season = 2020

In [30]:
#create the API  request with get request 
#CONSTRUCT THE API REQUEST 
url = 'https://free-api-live-football-data.p.rapidapi.com/football-get-standing-all' 
headers = {
    	"x-rapidapi-host" :"free-api-live-football-data.p.rapidapi.com",
	    "x-rapidapi-key": "d3335995damsh29951d1696ba593p1c1ad1jsn963fdedea7cd"
}
query_string = {
    "leagueid": league_id
   
}

#SEND THE API REQUEST 
response = requests.get(url,headers=headers, params=query_string)
payload = response.json()
print(payload)

file_name = f"league_{league_id}_standings.json"

#save JSON to a local file in Python
with open(file_name, "w",encoding="utf-8") as f:
    json.dump(payload, f, indent=2,ensure_ascii=False)

file_name = f"league_{league_id}_standings.json"


{'status': 'success', 'response': {'standing': [{'name': 'Arsenal', 'shortName': 'Arsenal', 'id': 9825, 'pageUrl': '/teams/9825/overview/arsenal', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 15, 'draws': 3, 'losses': 2, 'scoresStr': '40-14', 'goalConDiff': 26, 'pts': 48, 'idx': 1, 'qualColor': '#2AD572'}, {'name': 'Manchester City', 'shortName': 'Man City', 'id': 8456, 'pageUrl': '/teams/8456/overview/manchester-city', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 13, 'draws': 3, 'losses': 4, 'scoresStr': '44-18', 'goalConDiff': 26, 'pts': 42, 'idx': 2, 'qualColor': '#2AD572'}, {'name': 'Aston Villa', 'shortName': 'Aston Villa', 'id': 10252, 'pageUrl': '/teams/10252/overview/aston-villa', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 13, 'draws': 3, 'losses': 4, 'scoresStr': '33-24', 'goalConDiff': 9, 'pts': 42, 'idx': 3, 'qualColor': '#2AD572'}, {'name': 'Liverpool', 'shortName': 'Liverpool', 'id': 8650, 'pageUrl': '/teams/8650/overview/liverpool'

In [41]:
with open(file_name, "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
data.keys()
#data.values()

dict_values(['success', {'standing': [{'name': 'Arsenal', 'shortName': 'Arsenal', 'id': 9825, 'pageUrl': '/teams/9825/overview/arsenal', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 15, 'draws': 3, 'losses': 2, 'scoresStr': '40-14', 'goalConDiff': 26, 'pts': 48, 'idx': 1, 'qualColor': '#2AD572'}, {'name': 'Manchester City', 'shortName': 'Man City', 'id': 8456, 'pageUrl': '/teams/8456/overview/manchester-city', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 13, 'draws': 3, 'losses': 4, 'scoresStr': '44-18', 'goalConDiff': 26, 'pts': 42, 'idx': 2, 'qualColor': '#2AD572'}, {'name': 'Aston Villa', 'shortName': 'Aston Villa', 'id': 10252, 'pageUrl': '/teams/10252/overview/aston-villa', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 13, 'draws': 3, 'losses': 4, 'scoresStr': '33-24', 'goalConDiff': 9, 'pts': 42, 'idx': 3, 'qualColor': '#2AD572'}, {'name': 'Liverpool', 'shortName': 'Liverpool', 'id': 8650, 'pageUrl': '/teams/8650/overview/liverpool', 'deducti

In [52]:
#parsing the JSON to isloate data needed 
#parse top level
status = payload['status']
standings = data['response']['standing']
teams_count = len(standings)
print(status, teams_count,standings)

success 20 [{'name': 'Arsenal', 'shortName': 'Arsenal', 'id': 9825, 'pageUrl': '/teams/9825/overview/arsenal', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 15, 'draws': 3, 'losses': 2, 'scoresStr': '40-14', 'goalConDiff': 26, 'pts': 48, 'idx': 1, 'qualColor': '#2AD572'}, {'name': 'Manchester City', 'shortName': 'Man City', 'id': 8456, 'pageUrl': '/teams/8456/overview/manchester-city', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 13, 'draws': 3, 'losses': 4, 'scoresStr': '44-18', 'goalConDiff': 26, 'pts': 42, 'idx': 2, 'qualColor': '#2AD572'}, {'name': 'Aston Villa', 'shortName': 'Aston Villa', 'id': 10252, 'pageUrl': '/teams/10252/overview/aston-villa', 'deduction': None, 'ongoing': None, 'played': 20, 'wins': 13, 'draws': 3, 'losses': 4, 'scoresStr': '33-24', 'goalConDiff': 9, 'pts': 42, 'idx': 3, 'qualColor': '#2AD572'}, {'name': 'Liverpool', 'shortName': 'Liverpool', 'id': 8650, 'pageUrl': '/teams/8650/overview/liverpool', 'deduction': None, 'ongoing': None

In [ ]:
#parse nested json
teams = [team.get('name', 'not found') for team in standings ]
teams

['Arsenal',
 'Manchester City',
 'Aston Villa',
 'Liverpool',
 'Chelsea',
 'Manchester United',
 'Brentford',
 'Sunderland',
 'Newcastle United',
 'Brighton & Hove Albion',
 'Fulham',
 'Everton',
 'Tottenham Hotspur',
 'Crystal Palace',
 'AFC Bournemouth',
 'Leeds United',
 'Nottingham Forest',
 'West Ham United',
 'Burnley',
 'Wolverhampton Wanderers']

In [73]:
#top team with more than 40 pts
top_5_teams = [ team.get('name', 'not found') for team in standings if team['pts']> 40]
top_5_teams

['Arsenal', 'Manchester City', 'Aston Villa']

In [75]:
for team in standings[:2]:
    print("----------------------------")
    print("team_rank:", team['idx'])
    print("team:", team['name'])
    print("pts:", team['pts'])
    print('played:', team['played'])
    print('wins:', team['wins'])
    print('draws:', team['draws'])
    print('losses:', team['losses'])
    scores =  team['scoresStr'].split("-")
    print('goals_for:', int(scores[0]))
    print('goals_against:', int(scores[1]))
    print('goalConDiff:', team['goalConDiff'])
       

----------------------------
team_rank: 1
team: Arsenal
pts: 48
played: 20
wins: 15
draws: 3
losses: 2
goals_for: 40
goals_against: 14
goalConDiff: 26
----------------------------
team_rank: 2
team: Manchester City
pts: 42
played: 20
wins: 13
draws: 3
losses: 4
goals_for: 44
goals_against: 18
goalConDiff: 26


In [80]:
#loop through each team and build a list of reacords
records = []
for team in standings:
   
    scores =  team['scoresStr'].split("-")

    record = (
            team['idx'],
            team['name'],
            team['pts'],
            team['played'],
            team['wins'],
            team['draws'],
            team['losses'],
            int(scores[0]),      # goals_for
            int(scores[1]),      # goals_against
            team['goalConDiff']
        )
    records.append(record)   

In [83]:
columns = ['rank',
    'name', 'pts', 'played', 'wins', 'draws', 
    'losses', 'goals_for', 'goals_against', 'goals_diff'
]


In [89]:
#create a dataframe of reocrds tuple and list of column names 
df = pd.DataFrame(records, columns=columns)
df.head()

,rank,name,pts,played,wins,draws,losses,goals_for,goals_against,goals_diff
0,1,Arsenal,48,20,15,3,2,40,14,26
1,2,Manchester City,42,20,13,3,4,44,18,26
2,3,Aston Villa,42,20,13,3,4,33,24,9
3,4,Liverpool,34,20,10,4,6,32,28,4
4,5,Chelsea,31,20,8,7,5,33,22,11
